In [ ]:
# Load the neccessary libraries

import os
import polars as pl

In [2]:
# First we need to load the data on diagnoses

os.chdir("/home/alex/ews/diagnoses")

diagn_embed = pl.read_parquet("embed_diagnoses_updated_prevs.parquet")

# Discard the unneccesary columns

diagn_embed = diagn_embed.select([col for col in diagn_embed.columns if not col.startswith("diagn_embed")])

In [3]:
# Replace "NA" with "Ingen" in the Previous_Diagnoses column
diagn_embed = diagn_embed.with_columns(
    pl.col("Previous_Diagnoses").str.replace("NA", "Ingen").alias("Previous_Diagnoses")
)

In [5]:
# Now we load the data on procedures

os.chdir("/home/alex/ews/NEWS2_Evaluation")

procs_embed = pl.read_parquet("procs_full_june25.parquet")

In [ ]:
# Load the other metadata now (this dataframe contains ews scores, age, sex, mortality status, hospital, department name, ...)

os.chdir("/home/alex/ews/aggregated")

data = pl.read_parquet("ews_interventions_24_updated.parquet")

In [ ]:
# We fix the datetime variables

data = data.with_columns([
    (pl.col("recorded_time") - pl.duration(hours=1)).alias("recorded_time"),
    (pl.col("HOSP_DISCH_TIME") - pl.duration(hours=1)).dt.replace_time_zone(None).alias("HOSP_DISCH_TIME"),
    pl.col("deathDate").dt.replace_time_zone(None).alias("deathDate")
])

# Drop a column which takes too much size and is not needed

data = data.drop("Aggregated_Information")

In [10]:
# Now join the data (metadata + procedures)

data_final = data.join(procs_embed, on= ["PT_ID","CSN"], how="left")

In [ ]:
# Now we join the data with the diagnoses

data_final = data_final.join(diagn_embed, on= ["PT_ID","CSN"], how="left")     

In [ ]:
# Drop more columns not needed

data_final = data_final.drop(["Hemoglobin","Leukocytes","Trombocytes","Kreatinin", "ALAT", "LDH", "Albumin", "CRP", "Laktak_ab", "Troponin", "Laktat_vb"])

In [ ]:
# These are not needed

data_final = data_final.drop(["pca_0","pca_1","pca_2","pca_3","pca_4","pca_5","pca_6","pca_7",
                              "pca_8","pca_9","pca_10","pca_11","pca_12","pca_13","pca_14","pca_15","pca_16",
                              "pca_17","pca_18","pca_19","pca_20","pca_21","pca_22","pca_23","pca_24","pca_25","pca_26","pca_27","pca_28","pca_29"])

In [ ]:
##########################################################
########### Inspecting the blood test data ###############
##########################################################

os.chdir("/home/alex/ews/NEWS2_Evaluation")

blood_tests = pl.read_parquet("blood_tests_imputed.parquet")

In [ ]:
# Clean up the column names in blood_tests dataframe
# Remove the suffix starting from semicolon for columns from 6th position onward
# Special handling for Laktat columns

# Get current column names
old_columns = blood_tests.columns

# Create new column names with special handling for Laktat columns
new_columns = []
for i, col in enumerate(old_columns):
    if i < 5:  # Keep first 5 columns as is
        new_columns.append(col)
    elif col == "Laktat;P(aB)_imputed":
        new_columns.append("Laktat_ab")
    elif col == "Laktat;P(vB)_imputed":
        new_columns.append("Laktat_vb")
    elif ';' in col:  # For all other columns with semicolon, remove suffix
        new_columns.append(col.split(';')[0])
    else:  # For columns without semicolon, keep as is
        new_columns.append(col)

# Create a mapping dictionary for renaming
column_mapping = {old: new for old, new in zip(old_columns, new_columns)}

# Apply the renaming
blood_tests = blood_tests.rename(column_mapping)

print("Old column names:")
for i, col in enumerate(old_columns):
    if i >= 5:  # Only show columns that were changed
        print(f"  {col}")

print("\nNew column names:")
for i, col in enumerate(new_columns):
    if i >= 5:  # Only show columns that were changed  
        print(f"  {col}")

In [ ]:
# More modifications

current_columns = blood_tests.columns
final_columns = []

for col in current_columns:
    if col == "C-reaktivt protein [CRP]":
        final_columns.append("CRP")
    elif "[ALAT]" in col:
        final_columns.append(col.replace(" [ALAT]", ""))
    elif "[LDH]" in col:
        final_columns.append(col.replace(" [LDH]", ""))
    else:
        final_columns.append(col)

# Create mapping for final renaming
final_mapping = {old: new for old, new in zip(current_columns, final_columns)}

# Apply the final renaming
blood_tests = blood_tests.rename(final_mapping)

print("Final column renaming:")
for old, new in final_mapping.items():
    if old != new:  # Only show columns that changed
        print(f"  {old} → {new}")

print(f"\nFinal columns: {blood_tests.columns}")

In [ ]:
# Compute historical blood test averages for each patient at each recorded time
# For each recorded_time, get average of all blood tests that ended before that time

# First, let's identify the blood test value columns (excluding the first 5 metadata columns)
blood_test_columns = blood_tests.columns[5:]  # All columns from 6th onward
print(f"Blood test columns to average: {blood_test_columns}")

# Create a cross join to get all combinations of data_final records with blood_tests
# Then filter where blood_test_end < recorded_time
historical_tests = data_final.select(["PT_ID", "Identifier", "recorded_time"]).join(
    blood_tests.select(["PT_ID", "Blood_Test_End"] + blood_test_columns),
    on="PT_ID",
    how="inner"
).filter(
    pl.col("Blood_Test_End") < pl.col("recorded_time")
)

print(f"Shape of historical tests: {historical_tests.shape}")

# Group by PT_ID and recorded_time, then compute averages for each blood test
# We'll use mean() which automatically ignores null values
historical_averages = historical_tests.group_by(["PT_ID", "recorded_time"]).agg([
    pl.col(col).mean().alias(f"avg_{col}") for col in blood_test_columns
])

print(f"Shape of historical averages: {historical_averages.shape}")

# Join the historical averages back to data_final
data_final_with_blood_history = data_final.join(
    historical_averages,
    on=["PT_ID", "recorded_time"],
    how="left"
)

print(f"Shape of final dataset: {data_final_with_blood_history.shape}")
print(f"New columns added: {[col for col in data_final_with_blood_history.columns if col.startswith('avg_')]}")

# Display a sample to verify the results
data_final_with_blood_history.head()

In [ ]:
# Drop more unnessary variables for now

data_final_with_blood_history = data_final_with_blood_history.drop(["preds","weights",
                                                                    "Interventions_24",
                                                                    "Respirator_Start",
                                                                    "Respirator_End",
                                                                    "ITA_Start",
                                                                    "ITA_End",
                                                                    "ITA_Department",
                                                                    "Respiration_Num",
                                                                    "ITA_Indicator"])

In [ ]:
# Drop even more variables

data_final_with_blood_history = data_final_with_blood_history.drop("Interventions")

In [ ]:
# Rename the avg_Troponin T column

# Rename the avg_Troponin T column to avg_Troponin
data_final_with_blood_history = data_final_with_blood_history.rename({"avg_Troponin T": "avg_Troponin"})

print(f"Column renamed successfully. New columns: {data_final_with_blood_history.columns}")

In [ ]:
########################################################
############### Inspect the ITA data ###################
########################################################

ita = pl.read_parquet("intensive_care.parquet")

In [ ]:
# Create two new columns based on ITA data:
# 1. Previous ICU/Respiratory history (before recorded_time)
# 2. Early ICU/Respiratory in current admission (within 24 hours after recorded_time)

# First, let's create a column for previous ICU/respiratory history
# Check if patient had any ITA_Start or Respirator_Start before recorded_time

# Create a cross join to get all combinations of data_final records with ita records
previous_icu_check = data_final_with_blood_history.select(["PT_ID", "Identifier", "recorded_time"]).join(
    ita.select(["PT_ID", "ITA_Start", "Respirator_Start"]),
    on="PT_ID",
    how="inner"
).filter(
    # Check if either ITA_Start or Respirator_Start occurred before recorded_time
    (pl.col("ITA_Start") < pl.col("recorded_time")) | 
    (pl.col("Respirator_Start") < pl.col("recorded_time"))
)

# Get unique combinations of PT_ID and recorded_time that had previous ICU/respiratory
previous_icu_patients = previous_icu_check.select(["PT_ID", "recorded_time"]).unique()

# Add a flag column
previous_icu_patients = previous_icu_patients.with_columns(
    pl.lit(1).alias("previous_icu_respiratory")
)

# Join back to main dataframe
data_final_with_blood_history = data_final_with_blood_history.join(
    previous_icu_patients,
    on=["PT_ID", "recorded_time"],
    how="left"
).with_columns(
    pl.col("previous_icu_respiratory").fill_null(0)
)

print(f"Previous ICU/Respiratory history column added")

# Second, create a column for early ICU/respiratory in current admission
# Check if patient had any ITA_Start or Respirator_Start within 24 hours after recorded_time

# Calculate 24 hours after recorded_time
early_icu_check = data_final_with_blood_history.select(["PT_ID", "Identifier", "recorded_time"]).with_columns(
    (pl.col("recorded_time") + pl.duration(hours=24)).alias("recorded_time_plus_24h")
).join(
    ita.select(["PT_ID", "ITA_Start", "Respirator_Start"]),
    on="PT_ID",
    how="inner"
).filter(
    # Check if either ITA_Start or Respirator_Start occurred within 24 hours after recorded_time
    ((pl.col("ITA_Start") >= pl.col("recorded_time")) & (pl.col("ITA_Start") <= pl.col("recorded_time_plus_24h"))) |
    ((pl.col("Respirator_Start") >= pl.col("recorded_time")) & (pl.col("Respirator_Start") <= pl.col("recorded_time_plus_24h")))
)

# Get unique combinations that had early ICU/respiratory
early_icu_patients = early_icu_check.select(["PT_ID", "recorded_time"]).unique()

# Add a flag column
early_icu_patients = early_icu_patients.with_columns(
    pl.lit(1).alias("early_icu_respiratory_24h")
)

# Join back to main dataframe
data_final_with_blood_history = data_final_with_blood_history.join(
    early_icu_patients,
    on=["PT_ID", "recorded_time"],
    how="left"
).with_columns(
    pl.col("early_icu_respiratory_24h").fill_null(0)
)

print(f"Early ICU/Respiratory (24h) column added")
print(f"Final dataset shape: {data_final_with_blood_history.shape}")

# Show summary of the new columns
print("\nSummary of new columns:")
print(f"Previous ICU/Respiratory history: {data_final_with_blood_history['previous_icu_respiratory'].sum()} patients out of {data_final_with_blood_history.shape[0]} records")
print(f"Early ICU/Respiratory (24h): {data_final_with_blood_history['early_icu_respiratory_24h'].sum()} patients out of {data_final_with_blood_history.shape[0]} records")

# Display a sample to verify
data_final_with_blood_history.select(["PT_ID", "recorded_time", "previous_icu_respiratory", "early_icu_respiratory_24h"]).head(10)

In [58]:
# Save data

os.chdir("/home/alex/ews/NEWS2_Evaluation")

data_final_with_blood_history.write_parquet("data_final.parquet")